<a href="https://colab.research.google.com/github/aaronjuma/Deep-Learning-Test-Repo/blob/main/Text_Binary_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import re
import shutil
import string

from tensorflow.keras import layers
from tensorflow.keras import losses

In [4]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

84125825/84125825 [==============================] - 47s 1us/step


In [19]:
train_dir = os.path.join(dataset_dir, 'train')
delete_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(delete_dir)

In [20]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [21]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [22]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


In [23]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation),'')

In [24]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [25]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [26]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [27]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [28]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [29]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, 16),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])


In [30]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [31]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
625/625 [==============================] - 5s 8ms/step - loss: 0.6635 - binary_accuracy: 0.6916 - val_loss: 0.6133 - val_binary_accuracy: 0.7710
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.5468 - binary_accuracy: 0.8022 - val_loss: 0.4964 - val_binary_accuracy: 0.8218
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.4432 - binary_accuracy: 0.8458 - val_loss: 0.4190 - val_binary_accuracy: 0.8476
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3774 - binary_accuracy: 0.8651 - val_loss: 0.3728 - val_binary_accuracy: 0.8616
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3344 - binary_accuracy: 0.8800 - val_loss: 0.3444 - val_binary_accuracy: 0.8672
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3040 - binary_accuracy: 0.8881 - val_loss: 0.3257 - val_binary_accuracy: 0.8720
Epoch 7/10
625/625 [==============================] - 3s 5ms/ste

In [32]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

782/782 [==============================] - 5s 5ms/step - loss: 0.3100 - accuracy: 0.8731
0.8730800151824951


In [35]:
examples = [
  "The movie was great!",
  "The movie was okay.",
  "The movie was terrible..."
]

predictions = export_model.predict(examples)
print(f"First accuracy is {predictions[0]}")
print(f"Second accuracy is {predictions[1]}")
print(f"Third accuracy is {predictions[2]}")

1/1 [==============================] - 0s 81ms/step
First accuracy is [0.6205119]
Second accuracy is [0.44215244]
Third accuracy is [0.36143732]
